# KPI Augmentation for EV Transition Dataset
This notebook integrates KPI calculations from KPI_Lib.py to create a comprehensive dataset for visualization.

In [3]:
import pandas as pd
import numpy as np
from KPI_Lib import *

# Suppress warnings for cleaner output
import warnings
warnings.filterwarnings('ignore')

## 1. Load the Dataset

In [4]:
# Load the dataset
df = pd.read_csv("C:\\College\\Hackathons\\DataZen_Case_Study_Comp\\Data\\ev_transition_forecast_dataset.csv")

print(f"Dataset shape: {df.shape}")
print(f"\nColumns in dataset:")
print(df.columns.tolist())
print(f"\nFirst few rows:")
df.head()

Dataset shape: (216, 28)

Columns in dataset:
['state', 'year', 'vehicle_segment', 'ev_share', 'ice_share', 'total_registrations', 'conversion_pressure', 'ev_yoy_growth', 'ice_yoy_change', 'transition_index', 'ev_share_t-1', 'ev_share_t-2', 'transition_index_t-1', 'ev_yoy_growth_t-1', 'ice_yoy_change_t-1', 'infra_yoy_growth_t-1', 'fast_charger_index_t-1', 'stations_per_10k_vehicles_t-1', 'avg_ev_subsidy_rs', 'subsidy_yoy_change', 'fuel_price_rs_per_litre', 'fuel_price_yoy_change', 'avg_income_index', 'high_infra_flag', 'high_subsidy_flag', 'ev_share_t+1', 'ev_share_t+2', 'ev_share_t+3']

First few rows:


,state,year,vehicle_segment,ev_share,ice_share,total_registrations,conversion_pressure,ev_yoy_growth,ice_yoy_change,transition_index,ev_share_t-1,ev_share_t-2,transition_index_t-1,ev_yoy_growth_t-1,ice_yoy_change_t-1,infra_yoy_growth_t-1,fast_charger_index_t-1,stations_per_10k_vehicles_t-1,avg_ev_subsidy_rs,subsidy_yoy_change,fuel_price_rs_per_litre,fuel_price_yoy_change,avg_income_index,high_infra_flag,high_subsidy_flag,ev_share_t+1,ev_share_t+2,ev_share_t+3
0,Assam,2018,2W,0.128363,0.871637,23493651,0.147266,0.083071,0.046899,0.036173,0.124610,0.136990,-0.103506,-0.100896,0.002610,NaN,NaN,NaN,47568.093385,-0.081954,102.742996,0.002695,1.029572,0,0,0.132657,0.132353,0.142693
1,Assam,2019,2W,0.132657,0.867343,23220339,0.152947,0.021434,-0.016503,0.037937,0.128363,0.124610,0.036173,0.083071,0.046899,NaN,2.5200,0.191541,48173.076923,0.012718,101.455577,-0.012530,1.002808,1,0,0.132353,0.142693,0.131056
2,Assam,2020,2W,0.132353,0.867647,22893042,0.152543,-0.016355,-0.013750,-0.002605,0.132657,0.128363,0.037937,0.021434,-0.016503,1.362222,6.8032,0.457788,48475.609756,0.006280,102.821829,0.013467,0.991829,0,0,0.142693,0.131056,0.134436
3,Assam,2021,2W,0.142693,0.857307,24848329,0.166443,0.170206,0.072475,0.097731,0.132353,0.132657,-0.002605,-0.016355,-0.013750,-0.619003,1.5390,0.176910,46441.947566,-0.041952,102.977678,0.001516,0.997378,0,0,0.131056,0.134436,0.132942
4,Assam,2018,3W,0.124926,0.875074,22201524,0.142760,-0.095743,-0.017303,-0.078440,0.134307,0.131212,0.028103,0.059656,0.031553,NaN,NaN,NaN,45083.682008,-0.117967,102.194895,0.000835,0.996151,0,0,0.133567,0.132629,0.138105


## 2. Prepare Data for KPI Functions
The KPI functions expect specific column names. We need to create/map columns to match the expected format.

In [5]:
# Create a copy to preserve original data
df_kpi = df.copy()

# Calculate actual registration numbers from shares and total registrations
# ev_share and ice_share are proportions, total_registrations is the total
df_kpi['ev_vehicle_registrations'] = df_kpi['ev_share'] * df_kpi['total_registrations']
df_kpi['ice_vehicle_registrations'] = df_kpi['ice_share'] * df_kpi['total_registrations']

# For infrastructure metrics, we need to create charging_stations column
# Using stations_per_10k_vehicles_t-1 to back-calculate charging stations
# stations_per_10k = (stations / total_vehicles) * 10000
# Therefore: stations = (stations_per_10k * total_vehicles) / 10000
df_kpi['charging_stations'] = (df_kpi['stations_per_10k_vehicles_t-1'] * df_kpi['total_registrations']) / 10000
df_kpi['charging_stations'] = df_kpi['charging_stations'].fillna(0)

# Fast charger percentage - using fast_charger_index_t-1 as a proxy
# Normalize it to 0-100 scale if needed
if 'fast_charger_index_t-1' in df_kpi.columns:
    df_kpi['fast_charger_pct'] = df_kpi['fast_charger_index_t-1'] * 100
    df_kpi['fast_charger_pct'] = df_kpi['fast_charger_pct'].clip(0, 100).fillna(0)

# Policy-related columns
# We'll use avg_ev_subsidy_rs as is
# Create binary flags for road tax exemption and registration fee waiver
# Using high_subsidy_flag as a proxy for policy support
df_kpi['road_tax_exemption'] = df_kpi['high_subsidy_flag'].fillna(0).astype(int)
df_kpi['registration_fee_waiver'] = df_kpi['high_subsidy_flag'].fillna(0).astype(int)

# For policy_start_year, we'll estimate based on when subsidies first appeared
# This is a simplification - in real scenario, you'd have actual policy start dates
policy_starts = df_kpi.groupby('state')['year'].min().reset_index()
policy_starts.columns = ['state', 'policy_start_year']
df_kpi = df_kpi.merge(policy_starts, on='state', how='left')

print("Data preparation complete!")
print(f"\nNew columns created:")
print([col for col in df_kpi.columns if col not in df.columns])
print(f"\nDataset shape: {df_kpi.shape}")

Data preparation complete!

New columns created:
['ev_vehicle_registrations', 'ice_vehicle_registrations', 'charging_stations', 'fast_charger_pct', 'road_tax_exemption', 'registration_fee_waiver', 'policy_start_year']

Dataset shape: (216, 35)


## 3. Calculate KPIs
Now we'll apply the KPI functions from KPI_Lib.py to calculate all relevant metrics.

In [6]:
# KPI 1: EV Adoption Rate
print("Calculating KPI 1: EV Adoption Rate...")
df_kpi = calculate_ev_adoption_rate(
    df_kpi,
    ev_col='ev_vehicle_registrations',
    ice_col='ice_vehicle_registrations',
    output_col='ev_adoption_rate'
)
print("✓ EV Adoption Rate calculated")
print(f"  Range: {df_kpi['ev_adoption_rate'].min():.2f}% to {df_kpi['ev_adoption_rate'].max():.2f}%")

Calculating KPI 1: EV Adoption Rate...
✓ EV Adoption Rate calculated
  Range: 11.58% to 14.46%


In [7]:
# KPI 2: ICE to EV Conversion Rate
print("Calculating KPI 2: ICE to EV Conversion Rate...")
df_kpi = calculate_ice_to_ev_conversion_rate(
    df_kpi,
    ev_col='ev_vehicle_registrations',
    ice_col='ice_vehicle_registrations',
    groupby_cols=['state', 'vehicle_segment'],
    output_col='ice_to_ev_conversion_rate'
)
print("✓ ICE to EV Conversion Rate calculated")
print(f"  Range: {df_kpi['ice_to_ev_conversion_rate'].min():.2f}% to {df_kpi['ice_to_ev_conversion_rate'].max():.2f}%")

Calculating KPI 2: ICE to EV Conversion Rate...
✓ ICE to EV Conversion Rate calculated
  Range: -3.24% to 4.26%


In [8]:
# KPI 3: Year-over-Year Growth Rate
print("Calculating KPI 3: YoY Growth Rate...")
df_kpi = calculate_yoy_growth_rate(
    df_kpi,
    value_col='ev_vehicle_registrations',
    groupby_cols=['state', 'vehicle_segment'],
    output_col='yoy_growth_rate'
)
print("✓ YoY Growth Rate calculated")
print(f"  Range: {df_kpi['yoy_growth_rate'].min():.2f}% to {df_kpi['yoy_growth_rate'].max():.2f}%")

Calculating KPI 3: YoY Growth Rate...
✓ YoY Growth Rate calculated
  Range: -20.87% to 28.93%


In [9]:
# KPI 4: Market Share by Segment
print("Calculating KPI 4: Market Share by Segment...")
df_kpi = calculate_market_share_by_segment(
    df_kpi,
    segment_col='vehicle_segment',
    ev_col='ev_vehicle_registrations',
    groupby_cols=['state', 'year'],
    output_col='segment_market_share'
)
print("✓ Market Share by Segment calculated")
print(f"  Range: {df_kpi['segment_market_share'].min():.2f}% to {df_kpi['segment_market_share'].max():.2f}%")

Calculating KPI 4: Market Share by Segment...
✓ Market Share by Segment calculated
  Range: 28.07% to 39.11%


In [10]:
# KPI 5: Infrastructure Adequacy Ratio
print("Calculating KPI 5: Infrastructure Adequacy Ratio...")
df_kpi = calculate_infrastructure_adequacy_ratio(
    df_kpi,
    stations_col='charging_stations',
    ev_col='ev_vehicle_registrations',
    output_col='stations_per_1000_evs',
    benchmark=10.0
)
print("✓ Infrastructure Adequacy Ratio calculated")
print(f"  Range: {df_kpi['stations_per_1000_evs'].min():.2f} to {df_kpi['stations_per_1000_evs'].max():.2f} stations/1000 EVs")

Calculating KPI 5: Infrastructure Adequacy Ratio...
✓ Infrastructure Adequacy Ratio calculated
  Range: 0.00 to 0.66 stations/1000 EVs


In [11]:
# KPI 6: Policy Effectiveness Score
print("Calculating KPI 6: Policy Effectiveness Score...")
df_kpi = calculate_policy_effectiveness_score(
    df_kpi,
    subsidy_col='avg_ev_subsidy_rs',
    tax_exemption_col='road_tax_exemption',
    reg_waiver_col='registration_fee_waiver',
    adoption_col='ev_adoption_rate',
    output_score_col='policy_score',
    output_roi_col='policy_roi'
)
print("✓ Policy Effectiveness Score calculated")
print(f"  Policy Score Range: {df_kpi['policy_score'].min():.2f} to {df_kpi['policy_score'].max():.2f}")
print(f"  Policy ROI Range: {df_kpi['policy_roi'].min():.2f} to {df_kpi['policy_roi'].max():.2f}")

Calculating KPI 6: Policy Effectiveness Score...
✓ Policy Effectiveness Score calculated
  Policy Score Range: 4.36 to 9.59
  Policy ROI Range: 1.28 to 3.12


In [12]:
# KPI 7: Fast Charging Availability (if data available)
if 'fast_charger_pct' in df_kpi.columns:
    print("Calculating KPI 7: Fast Charging Availability...")
    df_kpi = calculate_fast_charging_availability(
        df_kpi,
        stations_col='charging_stations',
        fast_pct_col='fast_charger_pct',
        output_col='fast_charging_score'
    )
    print("✓ Fast Charging Availability calculated")
    print(f"  Range: {df_kpi['fast_charging_score'].min():.2f} to {df_kpi['fast_charging_score'].max():.2f}")
else:
    print("⚠ Skipping Fast Charging Availability (no fast_charger_pct column)")

Calculating KPI 7: Fast Charging Availability...
✓ Fast Charging Availability calculated
  Range: 0.00 to 2166.62


In [13]:
# KPI 8: Economic Viability Index (if data available)
if 'avg_income_index' in df_kpi.columns and 'fuel_price_rs_per_litre' in df_kpi.columns:
    print("Calculating KPI 8: Economic Viability Index...")
    df_kpi = calculate_economic_viability_index(
        df_kpi,
        income_col='avg_income_index',
        fuel_price_col='fuel_price_rs_per_litre',
        output_col='economic_viability_index'
    )
    print("✓ Economic Viability Index calculated")
    print(f"  Range: {df_kpi['economic_viability_index'].min():.2f} to {df_kpi['economic_viability_index'].max():.2f}")
else:
    print("⚠ Skipping Economic Viability Index (missing required columns)")

Calculating KPI 8: Economic Viability Index...
✓ Economic Viability Index calculated
  Range: 0.94 to 1.03


In [14]:
# KPI 9: Policy Maturity
if 'policy_start_year' in df_kpi.columns:
    print("Calculating KPI 9: Policy Maturity...")
    df_kpi = calculate_policy_maturity(
        df_kpi,
        current_year=2024,
        policy_start_col='policy_start_year',
        output_col='policy_age_years'
    )
    print("✓ Policy Maturity calculated")
    print(f"  Range: {df_kpi['policy_age_years'].min():.0f} to {df_kpi['policy_age_years'].max():.0f} years")
else:
    print("⚠ Skipping Policy Maturity (no policy_start_year column)")

Calculating KPI 9: Policy Maturity...
✓ Policy Maturity calculated
  Range: 6 to 6 years


In [15]:
# KPI 10: CAGR (Compound Annual Growth Rate)
print("Calculating KPI 10: CAGR...")
# Calculate CAGR for the available period
start_year = df_kpi['year'].min()
end_year = df_kpi['year'].max()

cagr_df = calculate_cagr(
    df_kpi,
    value_col='ev_vehicle_registrations',
    groupby_cols=['state', 'vehicle_segment'],
    start_year=start_year,
    end_year=end_year,
    output_col='cagr'
)

# Merge CAGR back to main dataframe
df_kpi = df_kpi.merge(
    cagr_df[['state', 'vehicle_segment', 'cagr']], 
    on=['state', 'vehicle_segment'], 
    how='left'
)

print("✓ CAGR calculated")
print(f"  Period: {start_year} to {end_year}")
print(f"  Range: {df_kpi['cagr'].min():.2f}% to {df_kpi['cagr'].max():.2f}%")

Calculating KPI 10: CAGR...
✓ CAGR calculated
  Period: 2018 to 2021
  Range: -7.50% to 8.64%


## 4. Summary of KPI Integration

In [16]:
# Display summary of all KPIs
print("="*80)
print("KPI INTEGRATION SUMMARY")
print("="*80)

kpi_columns = [
    'ev_adoption_rate',
    'ice_to_ev_conversion_rate',
    'yoy_growth_rate',
    'segment_market_share',
    'stations_per_1000_evs',
    'infrastructure_gap',
    'policy_score',
    'policy_roi',
    'cagr'
]

# Add optional KPIs if they exist
if 'fast_charging_score' in df_kpi.columns:
    kpi_columns.append('fast_charging_score')
if 'economic_viability_index' in df_kpi.columns:
    kpi_columns.append('economic_viability_index')
if 'policy_age_years' in df_kpi.columns:
    kpi_columns.append('policy_age_years')

# Filter to only existing columns
existing_kpi_cols = [col for col in kpi_columns if col in df_kpi.columns]

print(f"\nTotal KPIs calculated: {len(existing_kpi_cols)}")
print(f"\nKPI Statistics:")
print(df_kpi[existing_kpi_cols].describe().round(2))

print(f"\nDataset Information:")
print(f"  Total rows: {len(df_kpi):,}")
print(f"  Total columns: {len(df_kpi.columns)}")
print(f"  Original columns: {len(df.columns)}")
print(f"  New columns added: {len(df_kpi.columns) - len(df.columns)}")
print(f"  States: {df_kpi['state'].nunique()}")
print(f"  Years: {df_kpi['year'].min()} to {df_kpi['year'].max()}")
print(f"  Vehicle segments: {df_kpi['vehicle_segment'].unique().tolist()}")

KPI INTEGRATION SUMMARY

Total KPIs calculated: 12

KPI Statistics:
       ev_adoption_rate  ...  policy_age_years
count            216.00  ...             216.0
mean              13.24  ...               6.0
std                0.48  ...               0.0
min               11.58  ...               6.0
25%               12.91  ...               6.0
50%               13.24  ...               6.0
75%               13.53  ...               6.0
max               14.46  ...               6.0

[8 rows x 12 columns]

Dataset Information:
  Total rows: 216
  Total columns: 55
  Original columns: 28
  New columns added: 27
  States: 18
  Years: 2018 to 2021
  Vehicle segments: ['2W', '3W', '4W']


## 5. Preview Augmented Dataset

In [17]:
# Display sample of augmented data with key KPIs
display_cols = ['state', 'year', 'vehicle_segment', 'ev_adoption_rate', 
                'yoy_growth_rate', 'segment_market_share', 'policy_score', 
                'stations_per_1000_evs', 'infrastructure_status']

# Filter to only existing columns
display_cols = [col for col in display_cols if col in df_kpi.columns]

print("Sample of augmented dataset with KPIs:")
df_kpi[display_cols].head(10)

Sample of augmented dataset with KPIs:


,state,year,vehicle_segment,ev_adoption_rate,yoy_growth_rate,segment_market_share,policy_score,stations_per_1000_evs,infrastructure_status
0,Assam,2018,2W,12.836272,0.000000,34.508375,4.756809,0.000000,Severe Shortage
1,Assam,2019,2W,13.265728,2.143376,33.455952,4.817308,0.144388,Severe Shortage
2,Assam,2020,2W,13.235327,-1.635465,33.737565,4.847561,0.345884,Severe Shortage
3,Assam,2021,2W,14.269322,17.020598,36.271925,4.644195,0.123979,Severe Shortage
4,Assam,2018,3W,12.492557,0.000000,31.737243,4.508368,0.000000,Severe Shortage
5,Assam,2019,3W,13.356659,14.450960,34.476830,9.231660,0.151751,Severe Shortage
6,Assam,2020,3W,13.262924,-5.002643,33.576903,9.051230,0.337239,Severe Shortage
7,Assam,2021,3W,13.810517,6.324437,32.799580,9.111789,0.128979,Severe Shortage
8,Assam,2018,4W,13.186195,0.000000,33.754381,9.147059,0.000000,Severe Shortage
9,Assam,2019,4W,13.332106,0.090412,32.067219,4.702128,0.150882,Severe Shortage


## 6. Export Augmented Dataset

In [18]:
# Export the complete augmented dataset
output_path = "C:\\College\\Hackathons\\DataZen_Case_Study_Comp\\Data\\ev_transition_with_kpis.csv"
df_kpi.to_csv(output_path, index=False)
print(f"✅ Augmented dataset exported to: {output_path}")
print(f"   File size: {len(df_kpi):,} rows × {len(df_kpi.columns)} columns")

# Also export dashboard-ready files
print("\nExporting dashboard-ready files...")
export_kpis_for_dashboard(df_kpi, output_prefix='C:\\College\\Hackathons\\DataZen_Case_Study_Comp\\Data\\dashboard')

print("\n" + "="*80)
print("KPI INTEGRATION COMPLETE!")
print("="*80)
print("\nYou now have a comprehensive dataset ready for visualization with:")
print("  ✓ All original features")
print("  ✓ Calculated vehicle registration numbers")
print("  ✓ EV adoption metrics")
print("  ✓ Growth and conversion rates")
print("  ✓ Infrastructure adequacy indicators")
print("  ✓ Policy effectiveness scores")
print("  ✓ Economic viability indices")
print("  ✓ Market segmentation analysis")
print("\nReady for dashboard creation! 🚀")

✅ Augmented dataset exported to: C:\College\Hackathons\DataZen_Case_Study_Comp\Data\ev_transition_with_kpis.csv
   File size: 216 rows × 55 columns

Exporting dashboard-ready files...
✓ Saved: C:\College\Hackathons\DataZen_Case_Study_Comp\Data\dashboard_state_year.csv
✓ Saved: C:\College\Hackathons\DataZen_Case_Study_Comp\Data\dashboard_segment_year.csv
✓ Saved: C:\College\Hackathons\DataZen_Case_Study_Comp\Data\dashboard_latest_2021.csv
✅ All dashboard files exported!

KPI INTEGRATION COMPLETE!

You now have a comprehensive dataset ready for visualization with:
  ✓ All original features
  ✓ Calculated vehicle registration numbers
  ✓ EV adoption metrics
  ✓ Growth and conversion rates
  ✓ Infrastructure adequacy indicators
  ✓ Policy effectiveness scores
  ✓ Economic viability indices
  ✓ Market segmentation analysis

Ready for dashboard creation! 🚀


## 7. Data Validation & Quality Checks

In [19]:
# Check for missing values in key KPIs
print("Missing Values in KPIs:")
print(df_kpi[existing_kpi_cols].isnull().sum())

print("\nData Quality Checks:")
print(f"  ✓ EV + ICE shares sum to ~100%: {((df_kpi['ev_share'] + df_kpi['ice_share']) * 100).mean():.2f}%")
print(f"  ✓ Adoption rate matches ev_share: {(df_kpi['ev_adoption_rate'] - df_kpi['ev_share'] * 100).abs().mean():.4f} (should be ~0)")
print(f"  ✓ No negative values in registrations: {(df_kpi['ev_vehicle_registrations'] < 0).sum()} negative values")
print(f"  ✓ Segment shares sum to 100% per state-year: {df_kpi.groupby(['state', 'year'])['segment_market_share'].sum().mean():.2f}%")

print("\n✅ Data validation complete!")

Missing Values in KPIs:
ev_adoption_rate             0
ice_to_ev_conversion_rate    0
yoy_growth_rate              0
segment_market_share         0
stations_per_1000_evs        0
infrastructure_gap           0
policy_score                 0
policy_roi                   0
cagr                         0
fast_charging_score          0
economic_viability_index     0
policy_age_years             0
dtype: int64

Data Quality Checks:
  ✓ EV + ICE shares sum to ~100%: 100.00%
  ✓ Adoption rate matches ev_share: 0.0000 (should be ~0)
  ✓ No negative values in registrations: 0 negative values
  ✓ Segment shares sum to 100% per state-year: 100.00%

✅ Data validation complete!
